In [2]:
import pandas as pd

In [14]:
df = pd.read_csv('/home/ndo/bash_output/data_labels/varscan_labels.csv', sep=",", encoding='utf-8')
df

KeyboardInterrupt: 

In [ ]:
labels01= df['labels01'].astype('int')

In [17]:
labels01

csv_filename = '/home/ndo/bash_output/encoded_sequences/labels01.csv'
labels01.to_csv(csv_filename,index=False)


In [ ]:
flank=df['1']
flank

0          ACTTACATGACCGACATGGTATTTTTATTTGGCAGCGCTGCTCTAA...
1          GAAATGAGATATTTATGTAGTCTTAAGGTATCTTCTCACAAATTAT...
2          GGGTGACAGGGCATTATATCAGCCACTTATCAACAAACATTTCAGA...
3          ACTTTGCTCAATCTTCGCCTTTGGGTTCCTTTAGGTTTATAATGAA...
4          TTAGAGGTAACTTTGCTGAACTTGTGAAGAATGGTGTGCAAGGTCC...
                                 ...                        
5975095    CCCATGGCAGTGCGACAGCTGCTTCTTGGCAACTCCACCATGATCA...
5975096    TGAATGTTGACAGCACAGGAATCAGGTCATACGGCCCTCCCATCTC...
5975097    AGATTCAGGGCCTGCCCAGGGGCAGAGTGCCCCCAGGACTCGCCCT...
5975098    AGATTCAGGGCCTGCCCAGGGGCAGAGTGCCCCCAGGACTCGCCCT...
5975099    GCTCTTAGGGTGGCCCCTAAGTGAGGCTCCTCAGTGGCTCCAGCTA...
Name: 1, Length: 5975100, dtype: object

**let's try with k-mers encoding first**


In [ ]:
# imports
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.layers import LSTM, Bidirectional

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pickle
import gc

tf.config.list_physical_devices()

2024-03-21 10:12:23.187773: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-21 10:12:23.192539: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-21 10:12:23.253021: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-21 10:12:24.550145: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [ ]:
# # Tokenize DNA sequences into 4-mers
# tokenizer = Tokenizer(char_level=True)
# tokenizer.fit_on_texts(flank)
# sequences = tokenizer.texts_to_sequences(flank)


In [15]:
csv_filename = '/home/ndo/bash_output/encoded_sequences/4mers_encoded.csv'
df.to_csv(csv_filename,index=False)


In [3]:
df_read = pd.read_csv('/home/ndo/bash_output/encoded_sequences/4mers_encoded.csv')
labels01 = pd.read_csv( '/home/ndo/bash_output/encoded_sequences/labels01.csv')

In [9]:
df_read.head()

,Sequence
0,"[2, 4, 1, 1, 2, 4, 2, 1, 3, 2, 4, 4, 3, 2, 4, ..."
1,"[3, 2, 2, 2, 1, 3, 2, 3, 2, 1, 2, 1, 1, 1, 2, ..."
2,"[3, 3, 3, 1, 3, 2, 4, 2, 3, 3, 3, 4, 2, 1, 1, ..."
3,"[2, 4, 1, 1, 1, 3, 4, 1, 4, 2, 2, 1, 4, 1, 1, ..."
4,"[1, 1, 2, 3, 2, 3, 3, 1, 2, 2, 4, 1, 1, 1, 3, ..."


In [5]:
kmers = np.asarray(df_read).astype(int)
labels01 = np.asarray(labels01)

NameError: name 'np' is not defined

In [32]:
from sklearn.model_selection import train_test_split

# Assuming 'data' contains your dataset and 'labels' contain corresponding labels

# Splitting the data into 80% training and 20% test/validation combined
X_train, X_test_val, y_train, y_test_val = train_test_split(df, labels01, test_size=0.2, random_state=42)

# Splitting the test/validation data into 50% test and 50% validation
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=42)

print("Training set size:", len(X_train))
print("Validation set size:", len(X_val))
print("Test set size:", len(X_test))

Training set size: 4780080
Validation set size: 597510
Test set size: 597510


In [38]:
X_train

,Sequence
5610236,"[3, 1, 1, 2, 1, 1, 4, 1, 2, 4, 4, 2, 4, 4, 2, ..."
3838868,"[2, 2, 1, 4, 1, 3, 4, 1, 2, 1, 1, 1, 3, 2, 2, ..."
3334233,"[3, 2, 2, 2, 2, 1, 1, 1, 1, 2, 4, 1, 2, 1, 2, ..."
5636610,"[2, 2, 1, 3, 3, 2, 3, 3, 4, 2, 4, 1, 1, 1, 3, ..."
31859,"[4, 2, 3, 4, 1, 2, 4, 1, 3, 2, 2, 3, 4, 1, 1, ..."
...,...
1570006,"[1, 1, 4, 1, 3, 2, 1, 1, 2, 1, 1, 2, 3, 1, 4, ..."
2234489,"[3, 3, 2, 1, 1, 3, 1, 1, 1, 1, 2, 3, 2, 3, 4, ..."
4926484,"[4, 4, 2, 3, 3, 4, 1, 3, 3, 2, 3, 1, 3, 4, 2, ..."
4304572,"[4, 2, 2, 3, 2, 3, 2, 2, 2, 3, 2, 1, 1, 1, 4, ..."


In [36]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, SpatialDropout1D

# Assuming sequences and labels are already defined

# Convert sequences and labels to numpy arrays
# sequences = np.array(df_read)
# labels01 = np.array(labels01)

# # Split the dataset into training, validation, and testing sets
# # You can adjust the test_size and validation_split as per your requirements
# X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# Define LSTM model architecture
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_vector_length))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

# Compile the model with binary cross-entropy loss and accuracy metric
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print model summary
print(model.summary())

# Train the model
epochs = 10
batch_size = 64
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val), verbose=1)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_2             │ ?                      │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).